In [18]:
import torch
import warnings
import torch.nn as nn
from torch.nn import functional as F
warnings.filterwarnings("ignore")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

max_iters = 10000
learning_rate = 3e-4
vocab_size = len(chars)
eval_iters = 250
n_embed = 384
n_head = 4

cpu


In [8]:
with open ("wizard_of_oz.txt", 'r', encoding = 'utf8') as f:
    text = f.read()

print(len(text))

230467


In [9]:
chars = sorted(set(text))

In [10]:
print(len(chars))
chars[:10]

76


['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-']

In [11]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode("Hello")
decoded_hello = decode(encoded_hello)
print(encoded_hello)
print(decoded_hello)

[31, 53, 60, 60, 63]
Hello


In [13]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[: 100])

tensor([75, 27, 63, 66, 63, 68, 56, 73,  1, 49, 62, 52,  1, 68, 56, 53,  1, 46,
        57, 74, 49, 66, 52,  1, 57, 62,  1, 38, 74,  0,  0,  0,  1,  1, 24,  1,
        29, 49, 57, 68, 56, 54, 69, 60,  1, 41, 53, 51, 63, 66, 52,  1, 63, 54,
         1, 43, 56, 53, 57, 66,  1, 24, 61, 49, 74, 57, 62, 55,  1, 24, 52, 70,
        53, 62, 68, 69, 66, 53, 67,  0,  1,  1,  1,  1, 57, 62,  1, 49, 62,  1,
        44, 62, 52, 53, 66, 55, 66, 63, 69, 62])


In [14]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is ", context, "target is", target)

when input is  tensor([75]) target is tensor(27)
when input is  tensor([75, 27]) target is tensor(63)
when input is  tensor([75, 27, 63]) target is tensor(66)
when input is  tensor([75, 27, 63, 66]) target is tensor(63)
when input is  tensor([75, 27, 63, 66, 63]) target is tensor(68)
when input is  tensor([75, 27, 63, 66, 63, 68]) target is tensor(56)
when input is  tensor([75, 27, 63, 66, 63, 68, 56]) target is tensor(73)
when input is  tensor([75, 27, 63, 66, 63, 68, 56, 73]) target is tensor(1)


In [15]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

block_size = 8 #The length of each sequence (or "block") of data to be processed by the model
batch_size = 4 #The number of such sequences processed in one forward and backward pass (i.e., in parallel).

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[53,  1, 56, 49, 52,  1, 50, 53],
        [32,  0, 71, 63, 62, 52, 53, 66],
        [60, 60,  1, 67, 68, 49, 62, 52],
        [46, 57, 74, 49, 66, 52, 10,  0]])
targets:
tensor([[ 1, 56, 49, 52,  1, 50, 53, 54],
        [ 0, 71, 63, 62, 52, 53, 66,  1],
        [60,  1, 67, 68, 49, 62, 52, 57],
        [57, 74, 49, 66, 52, 10,  0,  0]])


In [16]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x



class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)  #This creates an embedding layer. It maps each token (word) in the vocabulary to a vector of size vocab_size
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) # no. of sequential layer example 4 decoder layer refer arch
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)   #read broadcasting semantic in pytorch
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape  #Batch size, sequence length, and number of classes (vocab size) respectively.
            logits = logits.view(B*T, C) #Reshapes the logits tensor to be of shape (B*T, C) for computing cross-entropy loss.
            targets = targets.view(B*T)  #Reshapes the targets tensor to match the shape required by F.cross_entropy.
            loss = F.cross_entropy(logits, targets)  #Computes the cross-entropy loss between the logits and the targets.
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)